In [1]:
import sys 
sys.path.append('../../../')
from plot_tools.fig_preamble import * # imports numpy, matplotlib, Py2PDF, plot_helper_funcs, plot_figstyle_sheet
import GPy

/home/riddhisw/anaconda2/lib/python2.7/site-packages/matplotlib/lines.py:1206: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self._markerfacecolor != fc:
/home/riddhisw/anaconda2/lib/python2.7/site-packages/matplotlib/lines.py:1182: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self._markeredgecolor != ec:


In [2]:
from gpr.common import get_data, simple_unlearnable_sine
from data_tools.load_raw_cluster_data import LoadExperiment as le
from plot_tools.plot_figstyle_sheet import color_pallete, predmarker, COLOURDICT, STYLEDICT

I also set up filepaths for saving data

In [3]:
ver=0
datapath = '../../DATA_v'+str(ver)+'_/' #v'+str(ver)+'/DATA_v'+str(ver)+'/'
datafile = 'GPR_kappa_demo2'

In [4]:
# Set experimental parameters
n_train = 2000
Delta_T = 0.001
n_predict = 150
n_testbefore=50

In [5]:
# Set initial starting values for L-BFGS-B OPtimiser in GPy
sigma_0 = 1.0
R_0 =  1.0

In [6]:
# Set length scale and periodicity initial values for L-BFGS-B OPtimiser in GPy
length_scale_0 = 3.0*Delta_T
period_0 = 2000.0

We now implement (a) manual constraints on noise variances and periodicities and (b) optimalised noise variances but constrained periodicities.

In [7]:
true_freq = (9., 10.)
noise_variances = ((sigma_0, R_0) , (10.0, 10.0))
noiselabel= ['No Opt.', 'Opt.']
periodicities = ((2000, 2001), (5000., 5001.))
offsets = [int(n_train- periodicities[idx_x][0]) for idx_x in xrange(len(periodicities))]


In [8]:
pred_list= np.zeros((len(true_freq),len(periodicities), len(noise_variances), n_predict+n_testbefore))
truth_list=np.zeros((len(true_freq),len(periodicities), len(noise_variances), n_predict+n_train))

In [9]:

for idx_freq in xrange(len(true_freq)): 

    X, Y, TestX, truth, msmts = simple_unlearnable_sine(nt=n_train, delta_t = Delta_T, 
                                                        f0 = true_freq[idx_freq], 
                                                        testpts=n_predict, randomise='y')
    
    for idx_per in xrange(len(periodicities)): 
       
        
        for idx_noise in xrange(len(noise_variances)): 
        
        
            # Reset
            kernel_per=0
            gauss=0
            exact=0
            m1=0
            
            # Store truth (even if its deterministic, helps with plotting)
            truth_list[idx_freq, idx_per, idx_noise, :] = truth
            
            # Built model
            kernel_per = GPy.kern.StdPeriodic(1, period=period_0, variance=sigma_0, 
                                              lengthscale=length_scale_0)
            gauss = GPy.likelihoods.Gaussian(variance=R_0)
            exact = GPy.inference.latent_function_inference.ExactGaussianInference()
            m1 = GPy.core.GP(X=X, Y=Y, kernel=kernel_per, likelihood=gauss, inference_method=exact)

            # Add constraints
            m1.std_periodic.period.constrain_bounded(periodicities[idx_per][0], periodicities[idx_per][1])
            if idx_noise != 0:
                m1.std_periodic.variance.constrain_bounded(0, noise_variances[idx_noise][0])
                m1.Gaussian_noise.variance.constrain_bounded(0, noise_variances[idx_noise][0])
            
            print("Commence GPRP for:", idx_freq, idx_per, idx_noise)
            print(m1)
            
            # Try optimise
            try:
                if idx_noise != 0:
                    m1.optimize(optimizer=None, messages=False)  # no pref optimiser
                pred_list[idx_freq, idx_per, idx_noise, :] = m1.predict(TestX)[0].flatten()
                print("Succeed - optimised params are")
                print(m1)
        
            except:
                print("Failed", idx_freq, idx_per, idx_noise)
        

Fourier resolution at training 0.5
True Frequency is 3.0


 ../../../gpr/common.py:97: VisibleDeprecationWarning:using a non-integer number instead of an integer will result in an error in the future
 ../../../gpr/common.py:98: VisibleDeprecationWarning:using a non-integer number instead of an integer will result in an error in the future


('Commence GPRP for:', 0, 0, 0)

Name : gp
Objective : 2346.85401697
Number of Parameters : 4
Number of Optimization Parameters : 4
Updates : True
Parameters:
  gp.                       |   value  |   constraints   |  priors
  std_periodic.variance     |     1.0  |       +ve       |        
  std_periodic.period       |  2000.0  |  2000.0,2001.0  |        
  std_periodic.lengthscale  |   0.003  |       +ve       |        
  Gaussian_noise.variance   |     1.0  |       +ve       |        
Succeed - optimised params are

Name : gp
Objective : 2346.85401697
Number of Parameters : 4
Number of Optimization Parameters : 4
Updates : True
Parameters:
  gp.                       |   value  |   constraints   |  priors
  std_periodic.variance     |     1.0  |       +ve       |        
  std_periodic.period       |  2000.0  |  2000.0,2001.0  |        
  std_periodic.lengthscale  |   0.003  |       +ve       |        
  Gaussian_noise.variance   |     1.0  |       +ve       |        
('Commence GP

In [10]:
np.savez(datapath+datafile, 
         pred_list=pred_list,
         truth_list=truth_list,
         n_train=n_train,
         n_predict=n_predict,
         n_testbefore=n_testbefore,
         sigma_0=sigma_0,
         R_0=R_0,
         true_freq=true_freq,
         noise_variances=noise_variances,
         periodicities=periodicities,
         offsets=offsets)